In [1]:
# 公式からとってくる。
# レースリスト、レースの参加メンバー、直前のオッズリストをESに突っ込む
# https://www.boatrace.jp/owpc/pc/race/index?hd=%s
# http://odds.kyotei24.jp/
# mySQL接続用のユーザ、パスワードは $BR_HOME/config_nogit/mysqlUser.json

import os
from bs4 import BeautifulSoup
import json
import urllib.request as u
import time
import re
import elasticsearch as es

import unicodedata

import pymysql

In [2]:
print(os.environ['GACRP_HOME'])
print(os.environ['BR_HOME'])

/home/ec2-user/kaggle/google-analytics-customer-revenue-prediction
/home/ec2-user/boatrace


In [3]:
# mySQLへの接続設定
with open(os.environ['BR_HOME']+'/config_nogit/mysqlUser.json') as mysqlconfig:
    mysqlConfig=json.load(mysqlconfig)

dbh=pymysql.connect(
    host='localhost',
    user=mysqlConfig['user'],
    password=mysqlConfig['password'],
    db='boatrace',
    charset='utf8',
    cursorclass=pymysql.cursors.DictCursor
)


In [4]:
#　会場とplaceIdとの紐づけファイルをロード。たぶん使わない。
with open('../config/def-place.json','r',encoding='utf8') as defPlace:
    placeDict=json.load(defPlace)
print(placeDict[0]['placeId'],placeDict[0]['placeName'])
#for i in range(len(placeDict)):
#    print("|"+placeDict[i]['placeId']+"|"+placeDict[i]['placeName']+"|")

01 桐生


In [5]:
def getRaceListKoushiki(ymd):
    html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/index?hd=%s" % ymd)
    time.sleep(1)
    soup = BeautifulSoup(html,"html.parser")
    for block in soup.select(".table1"):
        recs=block.select("tr")
        for rec in recs:
            tds=rec.select("td")
            if len(tds)>0:
                tmp=tds[0]
                #placePattern=r'tpl=(\d\d)'
                placePattern=r'tv(\d*)'
                placeId=re.search(placePattern,str(tmp)).group(1)
                #print(placeId)
                
                tmp=tds[2]
                gradePattern=r'class=\"is-(.*?)\"'
                grade=re.search(gradePattern,str(tmp)).group(1)
                #print(grade)
                
                rname=tds[4].string
                #print(rname)
                
                for i in range(1,12+1):
                    raceId=ymd+'-'+str(placeId).zfill(2)+'-'+str(i).zfill(2)
                    raceDate=ymd
                    raceNumber=i
                    raceName=rname
                    raceGrade=grade
                    racePlaceId=str(placeId).zfill(2)
                    
                    yield raceId,raceDate,raceNumber,raceName,raceGrade,racePlaceId

ymd = "20190321"
with dbh.cursor() as cursor:
    for ret in getRaceListKoushiki(ymd):
        #racemanagementをここでは整備する。
        sel_sql="SELECT count(*) FROM racemanagement WHERE raceid='%s'" % (ret[0])
        cursor.execute(sel_sql)
        results=cursor.fetchone()
        
        if results['count(*)']== 0: #なければINSERT
            ins_sql="INSERT INTO racemanagement (raceId,raceDate,raceNumber,raceGrade,racePlaceId,racebeforeinfo_flg,racemember_flg,raceodds3t_flg,raceodds3f_flg,raceodds2t_flg,raceodds2f_flg,raceodds1t_flg,raceodds1f_flg) VALUES ('%s' ,'%s',%d,'%s','%s',FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE)" % (ret[0],ret[1],ret[2],ret[4],ret[5])
            #print(ins_sql)
            cursor.execute(ins_sql)
    dbh.commit()

In [6]:
def getRaceMemberKoushiki(raceId):
    #html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/index?hd=%s" % ymd)
    raceDate=raceId[0:8]
    raceNumber=int(raceId[12:14])
    placeId=raceId[9:11]
    print(raceDate,raceNumber,placeId)
    html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/racelist?rno=%d&jcd=%s&hd=%s" %(raceNumber,placeId,raceDate) )
    time.sleep(1)
    soup = BeautifulSoup(html,"html.parser")
    for block in soup.select(".is-fs12"):
        #print(block)
        recs=block.select("tr")
        rec=recs[0]
        tds=rec.select("td")
        lane=unicodedata.normalize('NFKC', tds[0].string)        
        tobanId=((tds[2].select(".is-fs11"))[0].contents[0].strip())[0:4]
        rank=(tds[2].select("span"))[0].string
        #print("lane:",lane)
        #print("tobanId:",tobanId)
        #print("rank:",rank)
        #print(tds[2]) ※　他に取りたい項目があればここからとってくる
        
        yield lane,tobanId,rank

dbh.rollback()
raceDate="20190321"
with dbh.cursor() as cursor:
    #racemanagementで、racememberが未取得になっていたら取得してフラグをtrueにする。
    sel_sql = "SELECT raceId FROM racemanagement WHERE raceDate='%s' and racemember_flg = FALSE" % (raceDate)
    cursor.execute(sel_sql)
    raceIdList=cursor.fetchall()
    for raceIdhash in raceIdList:
        raceId = raceIdhash['raceId']
        print(raceId)        
        with dbh.cursor() as ins_cursor:
            for ret in getRaceMemberKoushiki(raceId):
                #print(ret)
                ins_sql="INSERT INTO racemember (raceId,rmLane,rmToban,rmRank) VALUES ('%s' ,%d,'%s','%s')" % (raceId,int(ret[0]),ret[1],ret[2])
                #print(ins_sql)
                ins_cursor.execute(ins_sql)
            upd_sql="UPDATE racemanagement SET racemember_flg = TRUE where raceId ='%s'" % (raceId)
            ins_cursor.execute(upd_sql)
            dbh.commit()


20190321-01-01
20190321 1 01
20190321-01-02
20190321 2 01
20190321-01-03
20190321 3 01
20190321-01-04
20190321 4 01
20190321-01-05
20190321 5 01
20190321-01-06
20190321 6 01
20190321-01-07
20190321 7 01
20190321-01-08
20190321 8 01
20190321-01-09
20190321 9 01
20190321-01-10
20190321 10 01
20190321-01-11
20190321 11 01
20190321-01-12
20190321 12 01
20190321-02-01
20190321 1 02
20190321-02-02
20190321 2 02
20190321-02-03
20190321 3 02
20190321-02-04
20190321 4 02
20190321-02-05
20190321 5 02
20190321-02-06
20190321 6 02
20190321-02-07
20190321 7 02
20190321-02-08
20190321 8 02
20190321-02-09
20190321 9 02
20190321-02-10
20190321 10 02
20190321-02-11
20190321 11 02
20190321-02-12
20190321 12 02
20190321-03-01
20190321 1 03
20190321-03-02
20190321 2 03
20190321-03-03
20190321 3 03
20190321-03-04
20190321 4 03
20190321-03-05
20190321 5 03
20190321-03-06
20190321 6 03
20190321-03-07
20190321 7 03
20190321-03-08
20190321 8 03
20190321-03-09
20190321 9 03
20190321-03-10
20190321 10 03
2019032

In [20]:
def getOldOddsKoushiki3rentan(raceId):
    raceDate=raceId[0:8]
    raceNumber=int(raceId[12:14])
    placeId=raceId[9:11]
    #print(raceDate,raceNumber,placeId)
    html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/odds3t?rno=%d&jcd=%s&hd=%s" %(raceNumber,placeId,raceDate) )
    time.sleep(1)
    soup = BeautifulSoup(html,"html.parser")
    
    # 3連単の組み合わせリストを辞書順に作成
    sanrentanList=[]
    for i in range(1,6+1):
        for j in range(1,6+1):
            for k in range(1,6+1):
                if (i==j or i==k or j==k):
                    continue
                else:
                    sanrentanList.append(str(i)+"-"+str(j)+"-"+str(k))
    sorted(sanrentanList)
    
    sanrentanOddsList=[]
    for oddspoint in soup.select(".oddsPoint"):
        sanrentanOddsList.append(oddspoint.string)
    
    cnt=0
    try:
        for i in range(6):
            for j in range(20):
                yield sanrentanList[i*20+j],sanrentanOddsList[i+j*6]
    except IndexError as e:
        print('[SKIP]IndexError発生 at 3連単オッズ取得　raceId:'+raceId )
                
    
#raceId="20190321-01-01"
raceDate="20190321"
dbh.rollback()

with dbh.cursor() as cursor:
    #racemanagementで3連単のオッズを埋める
    sel_sql = "SELECT raceId FROM racemanagement WHERE raceDate='%s' and raceodds3t_flg = FALSE" % (raceDate)
    cursor.execute(sel_sql)
    raceIdList=cursor.fetchall()
    for raceIdhash in raceIdList:
        raceId = raceIdhash['raceId']
        print(raceId)
        with dbh.cursor() as ins_cursor:
            for ret in getOldOddsKoushiki3rentan(raceId):
                #print(ret)
                if ret[1] == '欠場':
                    print('[SKIP]欠場 at 3連単オッズ取得 raceId:'+raceId)
                else:
                    ins_sql="INSERT INTO raceodds (raceId,oddsType,funaken,odds) VALUES ('%s' ,'3t','%s','%s')" % (raceId,ret[0],ret[1])
                    ins_cursor.execute(ins_sql)
            upd_sql = "UPDATE racemanagement SET raceodds3t_flg = TRUE where raceId ='%s'" % (raceId)
            ins_cursor.execute(upd_sql)
            dbh.commit()

20190321-08-10
('1-2-3', '欠場')
[SKIP]欠場 at 3連単オッズ取得 raceId:20190321-08-10
('1-2-4', '欠場')
[SKIP]欠場 at 3連単オッズ取得 raceId:20190321-08-10
('1-2-5', '欠場')
[SKIP]欠場 at 3連単オッズ取得 raceId:20190321-08-10
('1-2-6', '欠場')
[SKIP]欠場 at 3連単オッズ取得 raceId:20190321-08-10
('1-3-2', '欠場')
[SKIP]欠場 at 3連単オッズ取得 raceId:20190321-08-10
('1-3-4', '欠場')
[SKIP]欠場 at 3連単オッズ取得 raceId:20190321-08-10
('1-3-5', '欠場')
[SKIP]欠場 at 3連単オッズ取得 raceId:20190321-08-10
('1-3-6', '欠場')
[SKIP]欠場 at 3連単オッズ取得 raceId:20190321-08-10
('1-4-2', '欠場')
[SKIP]欠場 at 3連単オッズ取得 raceId:20190321-08-10
('1-4-3', '欠場')
[SKIP]欠場 at 3連単オッズ取得 raceId:20190321-08-10
('1-4-5', '欠場')
[SKIP]欠場 at 3連単オッズ取得 raceId:20190321-08-10
('1-4-6', '欠場')
[SKIP]欠場 at 3連単オッズ取得 raceId:20190321-08-10
('1-5-2', '欠場')
[SKIP]欠場 at 3連単オッズ取得 raceId:20190321-08-10
('1-5-3', '欠場')
[SKIP]欠場 at 3連単オッズ取得 raceId:20190321-08-10
('1-5-4', '欠場')
[SKIP]欠場 at 3連単オッズ取得 raceId:20190321-08-10
('1-5-6', '欠場')
[SKIP]欠場 at 3連単オッズ取得 raceId:20190321-08-10
('1-6-2', '欠場')
[SKIP]欠場 at 3連単オッズ取得 race

[SKIP]IndexError発生 at 3連単オッズ取得　raceId:20190321-10-01
20190321-10-02
[SKIP]IndexError発生 at 3連単オッズ取得　raceId:20190321-10-02
20190321-10-03
[SKIP]IndexError発生 at 3連単オッズ取得　raceId:20190321-10-03
20190321-10-04
[SKIP]IndexError発生 at 3連単オッズ取得　raceId:20190321-10-04
20190321-10-05
[SKIP]IndexError発生 at 3連単オッズ取得　raceId:20190321-10-05
20190321-10-06
[SKIP]IndexError発生 at 3連単オッズ取得　raceId:20190321-10-06
20190321-10-07
[SKIP]IndexError発生 at 3連単オッズ取得　raceId:20190321-10-07
20190321-10-08
[SKIP]IndexError発生 at 3連単オッズ取得　raceId:20190321-10-08
20190321-10-09
[SKIP]IndexError発生 at 3連単オッズ取得　raceId:20190321-10-09
20190321-10-10
[SKIP]IndexError発生 at 3連単オッズ取得　raceId:20190321-10-10
20190321-10-11
[SKIP]IndexError発生 at 3連単オッズ取得　raceId:20190321-10-11
20190321-10-12
[SKIP]IndexError発生 at 3連単オッズ取得　raceId:20190321-10-12
20190321-12-01
('1-2-3', '34.1')
('1-2-4', '15.7')
('1-2-5', '23.7')
('1-2-6', '49.3')
('1-3-2', '65.6')
('1-3-4', '59.8')
('1-3-5', '96.9')
('1-3-6', '156.8')
('1-4-2', '19.3')
('1-4-3', '44.4')
('1

('1-2-3', '17.6')
('1-2-4', '23.6')
('1-2-5', '19.7')
('1-2-6', '13.9')
('1-3-2', '20.0')
('1-3-4', '22.2')
('1-3-5', '14.7')
('1-3-6', '10.9')
('1-4-2', '32.1')
('1-4-3', '27.6')
('1-4-5', '24.6')
('1-4-6', '15.7')
('1-5-2', '24.6')
('1-5-3', '21.5')
('1-5-4', '33.2')
('1-5-6', '11.1')
('1-6-2', '18.8')
('1-6-3', '13.5')
('1-6-4', '20.5')
('1-6-5', '11.7')
('2-1-3', '288.9')
('2-1-4', '385.2')
('2-1-5', '379.7')
('2-1-6', '298.5')
('2-3-1', '537.2')
('2-3-4', '1384')
('2-3-5', '1460')
('2-3-6', '1361')
('2-4-1', '815.4')
('2-4-3', '2113')
('2-4-5', '1804')
('2-4-6', '1155')
('2-5-1', '803.1')
('2-5-3', '1912')
('2-5-4', '3346')
('2-5-6', '1207')
('2-6-1', '642.5')
('2-6-3', '1867')
('2-6-4', '1983')
('2-6-5', '1487')
('3-1-2', '277.4')
('3-1-4', '286.3')
('3-1-5', '211.9')
('3-1-6', '201.8')
('3-2-1', '368.4')
('3-2-4', '1434')
('3-2-5', '1529')
('3-2-6', '1409')
('3-4-1', '603.8')
('3-4-2', '2231')
('3-4-5', '1727')
('3-4-6', '1207')
('3-5-1', '710.7')
('3-5-2', '1765')
('3-5-4', '27

('1-2-3', '15.8')
('1-2-4', '12.8')
('1-2-5', '22.0')
('1-2-6', '54.8')
('1-3-2', '16.2')
('1-3-4', '9.1')
('1-3-5', '13.9')
('1-3-6', '34.5')
('1-4-2', '15.8')
('1-4-3', '12.9')
('1-4-5', '12.4')
('1-4-6', '39.6')
('1-5-2', '44.2')
('1-5-3', '34.8')
('1-5-4', '26.5')
('1-5-6', '79.0')
('1-6-2', '167.8')
('1-6-3', '114.6')
('1-6-4', '126.8')
('1-6-5', '166.2')
('2-1-3', '96.8')
('2-1-4', '107.6')
('2-1-5', '138.4')
('2-1-6', '236.4')
('2-3-1', '192.9')
('2-3-4', '510.9')
('2-3-5', '614.4')
('2-3-6', '814.9')
('2-4-1', '214.8')
('2-4-3', '578.3')
('2-4-5', '503.1')
('2-4-6', '752.7')
('2-5-1', '365.9')
('2-5-3', '986.1')
('2-5-4', '725.1')
('2-5-6', '1248')
('2-6-1', '1108')
('2-6-3', '1577')
('2-6-4', '2241')
('2-6-5', '2561')
('3-1-2', '58.6')
('3-1-4', '30.4')
('3-1-5', '40.4')
('3-1-6', '85.3')
('3-2-1', '145.8')
('3-2-4', '298.3')
('3-2-5', '389.7')
('3-2-6', '570.0')
('3-4-1', '49.1')
('3-4-2', '151.3')
('3-4-5', '66.7')
('3-4-6', '140.7')
('3-5-1', '96.9')
('3-5-2', '245.9')
('3-

('1-2-3', '8.8')
('1-2-4', '18.7')
('1-2-5', '79.6')
('1-2-6', '14.8')
('1-3-2', '4.7')
('1-3-4', '5.9')
('1-3-5', '52.8')
('1-3-6', '6.5')
('1-4-2', '36.1')
('1-4-3', '26.1')
('1-4-5', '153.8')
('1-4-6', '35.6')
('1-5-2', '227.4')
('1-5-3', '160.4')
('1-5-4', '279.4')
('1-5-6', '153.3')
('1-6-2', '23.5')
('1-6-3', '14.3')
('1-6-4', '39.9')
('1-6-5', '78.8')
('2-1-3', '55.3')
('2-1-4', '98.0')
('2-1-5', '253.0')
('2-1-6', '72.4')
('2-3-1', '137.6')
('2-3-4', '348.9')
('2-3-5', '1019')
('2-3-6', '248.5')
('2-4-1', '304.0')
('2-4-3', '585.4')
('2-4-5', '1816')
('2-4-6', '532.6')
('2-5-1', '1124')
('2-5-3', '2322')
('2-5-4', '2833')
('2-5-6', '1491')
('2-6-1', '254.8')
('2-6-3', '343.9')
('2-6-4', '665.2')
('2-6-5', '1180')
('3-1-2', '55.3')
('3-1-4', '60.4')
('3-1-5', '231.1')
('3-1-6', '45.0')
('3-2-1', '144.5')
('3-2-4', '387.1')
('3-2-5', '1049')
('3-2-6', '283.3')
('3-4-1', '160.2')
('3-4-2', '451.2')
('3-4-5', '963.8')
('3-4-6', '360.5')
('3-5-1', '833.4')
('3-5-2', '2114')
('3-5-4'

...

('1-2-3', '11.7')
('1-2-4', '30.3')
('1-2-5', '15.6')
('1-2-6', '8.6')
('1-3-2', '31.3')
('1-3-4', '99.2')
('1-3-5', '70.4')
('1-3-6', '42.7')
('1-4-2', '92.0')
('1-4-3', '168.7')
('1-4-5', '183.3')
('1-4-6', '132.9')
('1-5-2', '60.9')
('1-5-3', '123.1')
('1-5-4', '194.4')
('1-5-6', '81.5')
('1-6-2', '18.7')
('1-6-3', '39.8')
('1-6-4', '76.2')
('1-6-5', '42.8')
('2-1-3', '19.6')
('2-1-4', '46.8')
('2-1-5', '21.0')
('2-1-6', '12.3')
('2-3-1', '57.3')
('2-3-4', '180.6')
('2-3-5', '88.3')
('2-3-6', '57.7')
('2-4-1', '162.6')
('2-4-3', '353.2')
('2-4-5', '261.4')
('2-4-6', '131.3')
('2-5-1', '70.1')
('2-5-3', '128.9')
('2-5-4', '280.0')
('2-5-6', '55.5')
('2-6-1', '26.6')
('2-6-3', '53.8')
('2-6-4', '83.7')
('2-6-5', '43.4')
('3-1-2', '118.2')
('3-1-4', '280.9')
('3-1-5', '192.7')
('3-1-6', '132.9')
('3-2-1', '121.2')
('3-2-4', '306.9')
('3-2-5', '167.0')
('3-2-6', '125.0')
('3-4-1', '521.3')
('3-4-2', '681.4')
('3-4-5', '848.4')
('3-4-6', '670.9')
('3-5-1', '412.1')
('3-5-2', '397.0')
('3

('1-2-3', '14.2')
('1-2-4', '21.8')
('1-2-5', '164.3')
('1-2-6', '31.8')
('1-3-2', '14.5')
('1-3-4', '16.5')
('1-3-5', '95.1')
('1-3-6', '19.3')
('1-4-2', '36.8')
('1-4-3', '32.7')
('1-4-5', '166.1')
('1-4-6', '43.5')
('1-5-2', '440.8')
('1-5-3', '285.7')
('1-5-4', '484.3')
('1-5-6', '398.5')
('1-6-2', '67.0')
('1-6-3', '55.0')
('1-6-4', '70.9')
('1-6-5', '269.3')
('2-1-3', '29.4')
('2-1-4', '37.4')
('2-1-5', '197.0')
('2-1-6', '46.3')
('2-3-1', '44.8')
('2-3-4', '38.4')
('2-3-5', '208.5')
('2-3-6', '38.6')
('2-4-1', '79.7')
('2-4-3', '60.1')
('2-4-5', '367.3')
('2-4-6', '63.4')
('2-5-1', '818.2')
('2-5-3', '613.6')
('2-5-4', '947.4')
('2-5-6', '892.5')
('2-6-1', '120.5')
('2-6-3', '76.9')
('2-6-4', '99.8')
('2-6-5', '433.7')
('3-1-2', '38.4')
('3-1-4', '32.8')
('3-1-5', '147.3')
('3-1-6', '36.6')
('3-2-1', '51.8')
('3-2-4', '37.5')
('3-2-5', '184.3')
('3-2-6', '35.3')
('3-4-1', '39.5')
('3-4-2', '37.6')
('3-4-5', '130.4')
('3-4-6', '26.3')
('3-5-1', '394.1')
('3-5-2', '467.5')
('3-5-4

('1-2-3', '24.1')
('1-2-4', '11.4')
('1-2-5', '127.9')
('1-2-6', '28.2')
('1-3-2', '31.8')
('1-3-4', '17.6')
('1-3-5', '119.9')
('1-3-6', '39.7')
('1-4-2', '8.7')
('1-4-3', '12.3')
('1-4-5', '65.5')
('1-4-6', '9.0')
('1-5-2', '324.7')
('1-5-3', '306.5')
('1-5-4', '232.7')
('1-5-6', '396.4')
('1-6-2', '57.6')
('1-6-3', '70.2')
('1-6-4', '31.8')
('1-6-5', '270.2')
('2-1-3', '88.2')
('2-1-4', '43.5')
('2-1-5', '395.2')
('2-1-6', '108.4')
('2-3-1', '212.2')
('2-3-4', '262.3')
('2-3-5', '944.9')
('2-3-6', '420.3')
('2-4-1', '67.6')
('2-4-3', '192.7')
('2-4-5', '634.7')
('2-4-6', '104.9')
('2-5-1', '1256')
('2-5-3', '1632')
('2-5-4', '1698')
('2-5-6', '2094')
('2-6-1', '237.1')
('2-6-3', '546.4')
('2-6-4', '204.3')
('2-6-5', '1963')
('3-1-2', '165.8')
('3-1-4', '87.8')
('3-1-5', '498.7')
('3-1-6', '204.6')
('3-2-1', '244.9')
('3-2-4', '319.7')
('3-2-5', '937.9')
('3-2-6', '508.8')
('3-4-1', '115.4')
('3-4-2', '255.4')
('3-4-5', '766.3')
('3-4-6', '228.0')
('3-5-1', '959.3')
('3-5-2', '1444')

('1-2-3', '118.9')
('1-2-4', '169.8')
('1-2-5', '382.7')
('1-2-6', '88.2')
('1-3-2', '220.3')
('1-3-4', '305.2')
('1-3-5', '427.7')
('1-3-6', '222.1')
('1-4-2', '291.0')
('1-4-3', '412.3')
('1-4-5', '615.6')
('1-4-6', '275.9')
('1-5-2', '1016')
('1-5-3', '783.3')
('1-5-4', '1035')
('1-5-6', '751.0')
('1-6-2', '96.6')
('1-6-3', '189.2')
('1-6-4', '216.1')
('1-6-5', '382.7')
('2-1-3', '123.5')
('2-1-4', '173.6')
('2-1-5', '325.2')
('2-1-6', '80.7')
('2-3-1', '111.6')
('2-3-4', '25.6')
('2-3-5', '77.8')
('2-3-6', '13.1')
('2-4-1', '171.4')
('2-4-3', '40.6')
('2-4-5', '121.2')
('2-4-6', '21.9')
('2-5-1', '617.4')
('2-5-3', '235.5')
('2-5-4', '330.6')
('2-5-6', '126.9')
('2-6-1', '50.8')
('2-6-3', '9.8')
('2-6-4', '14.3')
('2-6-5', '43.8')
('3-1-2', '348.0')
('3-1-4', '516.7')
('3-1-5', '631.6')
('3-1-6', '301.0')
('3-2-1', '222.5')
('3-2-4', '60.3')
('3-2-5', '175.8')
('3-2-6', '26.4')
('3-4-1', '384.1')
('3-4-2', '95.9')
('3-4-5', '244.7')
('3-4-6', '70.6')
('3-5-1', '738.3')
('3-5-2', '4

('1-2-3', '14.5')
('1-2-4', '10.7')
('1-2-5', '31.1')
('1-2-6', '29.2')
('1-3-2', '17.9')
('1-3-4', '13.2')
('1-3-5', '25.1')
('1-3-6', '30.6')
('1-4-2', '10.7')
('1-4-3', '13.4')
('1-4-5', '23.4')
('1-4-6', '15.3')
('1-5-2', '65.6')
('1-5-3', '61.9')
('1-5-4', '55.2')
('1-5-6', '89.7')
('1-6-2', '72.2')
('1-6-3', '80.1')
('1-6-4', '49.6')
('1-6-5', '125.8')
('2-1-3', '53.1')
('2-1-4', '43.4')
('2-1-5', '100.4')
('2-1-6', '90.7')
('2-3-1', '110.4')
('2-3-4', '167.4')
('2-3-5', '269.4')
('2-3-6', '269.7')
('2-4-1', '70.4')
('2-4-3', '153.3')
('2-4-5', '220.4')
('2-4-6', '157.2')
('2-5-1', '281.9')
('2-5-3', '487.5')
('2-5-4', '420.4')
('2-5-6', '567.3')
('2-6-1', '325.6')
('2-6-3', '608.4')
('2-6-4', '433.6')
('2-6-5', '878.6')
('3-1-2', '81.4')
('3-1-4', '52.9')
('3-1-5', '107.8')
('3-1-6', '118.1')
('3-2-1', '136.8')
('3-2-4', '181.8')
('3-2-5', '342.7')
('3-2-6', '326.5')
('3-4-1', '84.2')
('3-4-2', '152.9')
('3-4-5', '176.6')
('3-4-6', '157.7')
('3-5-1', '242.4')
('3-5-2', '482.5')


In [21]:
def getOldOddsKoushiki3renfuku(raceId):
    raceDate=raceId[0:8]
    raceNumber=int(raceId[12:14])
    placeId=raceId[9:11]
    print(raceDate,raceNumber,placeId)
    html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/odds3f?rno=%d&jcd=%s&hd=%s" %(raceNumber,placeId,raceDate) )
    time.sleep(1)
    soup = BeautifulSoup(html,"html.parser")
    
    # 3連複の組み合わせリストをサイトからの取得可能順に作成
    sanrenfukuList=[]
    for k in range(3,6+1):
        sanrenfukuList.append(str(1)+"-"+str(2)+"-"+str(k))
    for k in range(4,6+1):
        for i in range(1,2+1):
            sanrenfukuList.append(str(i)+"-"+str(3)+"-"+str(k))
    for k in range(5,6+1):
        for i in range(1,3+1):
            sanrenfukuList.append(str(i)+"-"+str(4)+"-"+str(k))
    for i in range(1,4+1):
        sanrenfukuList.append(str(i)+"-"+str(5)+"-"+str(6))
    
    #print(sanrenfukuList)
    sanrenfukuOddsList=[]
    for oddspoint in soup.select(".oddsPoint"):
        sanrenfukuOddsList.append(oddspoint.string)
    
    try:
        for i in range(len(sanrenfukuList)):
            yield sanrenfukuList[i],sanrenfukuOddsList[i]
    except IndexError as e:
        print('[SKIP]IndexError発生 at 3連複オッズ取得 raceId:'+raceId)
    
#raceId="20190321-01-01"
raceDate="20190321"
dbh.rollback()

with dbh.cursor() as cursor:
    #racemanagementで3連複のオッズを埋める
    sel_sql = "SELECT raceId FROM racemanagement WHERE raceDate='%s' and raceodds3f_flg = FALSE" % (raceDate)
    cursor.execute(sel_sql)
    raceIdList=cursor.fetchall()
    for raceIdhash in raceIdList:
        raceId = raceIdhash['raceId']
        print(raceId)
        with dbh.cursor() as ins_cursor:
            for ret in getOldOddsKoushiki3renfuku(raceId):
                #print(ret)
                if ret[1] == '欠場':
                    print('[SKIP]欠場 at 3連複オッズ取得 raceId:'+raceId)
                else:
                    ins_sql="INSERT INTO raceodds (raceId,oddsType,funaken,odds) VALUES ('%s' ,'3f','%s','%s')" % (raceId,ret[0],ret[1])
                    ins_cursor.execute(ins_sql)
            upd_sql = "UPDATE racemanagement SET raceodds3f_flg = TRUE where raceId ='%s'" % (raceId)
            ins_cursor.execute(upd_sql)
            dbh.commit()



20190321-01-01
20190321 1 01
20190321-01-02
20190321 2 01
20190321-01-03
20190321 3 01
20190321-01-04
20190321 4 01
20190321-01-05
20190321 5 01
20190321-01-06
20190321 6 01
20190321-01-07
20190321 7 01
20190321-01-08
20190321 8 01
20190321-01-09
20190321 9 01
20190321-01-10
20190321 10 01
20190321-01-11
20190321 11 01
20190321-01-12
20190321 12 01
20190321-02-01
20190321 1 02
20190321-02-02
20190321 2 02
20190321-02-03
20190321 3 02
20190321-02-04
20190321 4 02
20190321-02-05
20190321 5 02
20190321-02-06
20190321 6 02
20190321-02-07
20190321 7 02
20190321-02-08
20190321 8 02
20190321-02-09
20190321 9 02
20190321-02-10
20190321 10 02
20190321-02-11
20190321 11 02
20190321-02-12
20190321 12 02
20190321-03-01
20190321 1 03
[SKIP]IndexError発生 at 3連複オッズ取得 raceId:20190321-03-01
20190321-03-02
20190321 2 03
[SKIP]IndexError発生 at 3連複オッズ取得 raceId:20190321-03-02
20190321-03-03
20190321 3 03
[SKIP]IndexError発生 at 3連複オッズ取得 raceId:20190321-03-03
20190321-03-04
20190321 4 03
[SKIP]IndexError発生 at 3

In [23]:
def getOldOddsKoushiki2rentan(raceId):
    raceDate=raceId[0:8]
    raceNumber=int(raceId[12:14])
    placeId=raceId[9:11]
    print(raceDate,raceNumber,placeId)
    html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/odds2tf?rno=%d&jcd=%s&hd=%s" %(raceNumber,placeId,raceDate) )
    time.sleep(1)
    soup = BeautifulSoup(html,"html.parser")
    
    try:
        block =(soup.select(".is-p3-0"))[0]
        tmpcnt1=1
        for trs in block.select("tr"):
            tmpcnt2=0
            tmp1stlane=1
            for tds in trs.select("td"):
                if tmpcnt2%2==0:
                    cs=tds.string
                else:
                    odds=tds.string
                    yield str(tmp1stlane)+"-"+str(cs),odds
                    tmp1stlane+=1
                tmpcnt2+=1
            tmpcnt1+=1
    except IndexError as e:
        print('[SKIP]IndexError発生 at 2連単オッズ取得 raceId:'+raceId)
        
#raceId="20190321-01-01"
raceDate="20190321"
dbh.rollback()

with dbh.cursor() as cursor:
    #racemanagementで2連単のオッズを埋める
    sel_sql = "SELECT raceId FROM racemanagement WHERE raceDate='%s' and raceodds2t_flg = FALSE" % (raceDate)
    cursor.execute(sel_sql)
    raceIdList=cursor.fetchall()
    for raceIdhash in raceIdList:
        raceId = raceIdhash['raceId']
        print(raceId)
        with dbh.cursor() as ins_cursor:
            for ret in getOldOddsKoushiki2rentan(raceId):
                #print(ret)
                if ret[1] == '欠場':
                    print('[SKIP]欠場 at 2連単オッズ取得 raceId:'+raceId)
                else:
                    ins_sql="INSERT INTO raceodds (raceId,oddsType,funaken,odds) VALUES ('%s' ,'2t','%s','%s')" % (raceId,ret[0],ret[1])
                    ins_cursor.execute(ins_sql)
            upd_sql = "UPDATE racemanagement SET raceodds2t_flg = TRUE where raceId ='%s'" % (raceId)
            ins_cursor.execute(upd_sql)
            dbh.commit()
    


20190321-03-01
20190321 1 03
indexerror?20190321-03-01
20190321-03-02
20190321 2 03
indexerror?20190321-03-02
20190321-03-03
20190321 3 03
indexerror?20190321-03-03
20190321-03-04
20190321 4 03
indexerror?20190321-03-04
20190321-03-05
20190321 5 03
indexerror?20190321-03-05
20190321-03-06
20190321 6 03
indexerror?20190321-03-06
20190321-03-07
20190321 7 03
indexerror?20190321-03-07
20190321-03-08
20190321 8 03
indexerror?20190321-03-08
20190321-03-09
20190321 9 03
indexerror?20190321-03-09
20190321-03-10
20190321 10 03
indexerror?20190321-03-10
20190321-03-11
20190321 11 03
indexerror?20190321-03-11
20190321-03-12
20190321 12 03
indexerror?20190321-03-12
20190321-05-01
20190321 1 05
20190321-05-02
20190321 2 05
20190321-05-03
20190321 3 05
20190321-05-04
20190321 4 05
20190321-05-05
20190321 5 05
20190321-05-06
20190321 6 05
20190321-05-07
20190321 7 05
20190321-05-08
20190321 8 05
20190321-05-09
20190321 9 05
20190321-05-10
20190321 10 05
20190321-05-11
20190321 11 05
20190321-05-12
2

In [24]:
def getOldOddsKoushiki2renfuku(raceId):
    raceDate=raceId[0:8]
    raceNumber=int(raceId[12:14])
    placeId=raceId[9:11]
    print(raceDate,raceNumber,placeId)
    html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/odds2tf?rno=%d&jcd=%s&hd=%s" %(raceNumber,placeId,raceDate) )
    time.sleep(1)
    soup = BeautifulSoup(html,"html.parser")
    
    try:
        block =(soup.select(".is-p3-0"))[1]
        tmpcnt1=1
        for trs in block.select("tr"):
            tmpcnt2=0
            tmp1stlane=1
            for tds in trs.select("td"):
                if tmpcnt2%2==0:
                    cs=tds.string
                elif tds['class'][0] == "oddsPoint":
                    odds=tds.string
                    yield str(tmp1stlane)+"-"+str(cs),odds
                    tmp1stlane+=1
                tmpcnt2+=1
            tmpcnt1+=1
    except IndexError as e:
        print('[SKIP]IndexError発生 at 2連複オッズ取得　raceId:'+raceId)
        
#raceId="20190321-01-01"
#for ret in getOldOddsKoushiki2renfuku(raceId):
#    print(ret)
    
raceDate="20190321"
dbh.rollback()

with dbh.cursor() as cursor:
    #racemanagementで2連複のオッズを埋める
    sel_sql = "SELECT raceId FROM racemanagement WHERE raceDate='%s' and raceodds2f_flg = FALSE" % (raceDate)
    cursor.execute(sel_sql)
    raceIdList=cursor.fetchall()
    for raceIdhash in raceIdList:
        raceId = raceIdhash['raceId']
        print(raceId)
        with dbh.cursor() as ins_cursor:
            for ret in getOldOddsKoushiki2renfuku(raceId):
                #print(ret)
                if ret[1] == '欠場':
                    print('[SKIP]欠場 at 2連複オッズ取得 raceId:'+raceId)
                else:
                    ins_sql="INSERT INTO raceodds (raceId,oddsType,funaken,odds) VALUES ('%s' ,'2f','%s','%s')" % (raceId,ret[0],ret[1])
                    ins_cursor.execute(ins_sql)
            upd_sql = "UPDATE racemanagement SET raceodds2f_flg = TRUE where raceId ='%s'" % (raceId)
            ins_cursor.execute(upd_sql)
            dbh.commit()



20190321-01-01
20190321 1 01
20190321-01-02
20190321 2 01
20190321-01-03
20190321 3 01
20190321-01-04
20190321 4 01
20190321-01-05
20190321 5 01
20190321-01-06
20190321 6 01
20190321-01-07
20190321 7 01
20190321-01-08
20190321 8 01
20190321-01-09
20190321 9 01
20190321-01-10
20190321 10 01
20190321-01-11
20190321 11 01
20190321-01-12
20190321 12 01
20190321-02-01
20190321 1 02
20190321-02-02
20190321 2 02
20190321-02-03
20190321 3 02
20190321-02-04
20190321 4 02
20190321-02-05
20190321 5 02
20190321-02-06
20190321 6 02
20190321-02-07
20190321 7 02
20190321-02-08
20190321 8 02
20190321-02-09
20190321 9 02
20190321-02-10
20190321 10 02
20190321-02-11
20190321 11 02
20190321-02-12
20190321 12 02
20190321-03-01
20190321 1 03
[SKIP]IndexError発生 at 2連複オッズ取得　raceId:20190321-03-01
20190321-03-02
20190321 2 03
[SKIP]IndexError発生 at 2連複オッズ取得　raceId:20190321-03-02
20190321-03-03
20190321 3 03
[SKIP]IndexError発生 at 2連複オッズ取得　raceId:20190321-03-03
20190321-03-04
20190321 4 03
[SKIP]IndexError発生 at 2

In [25]:
def getOldOddsKoushikiTansho(raceId):
    raceDate=raceId[0:8]
    raceNumber=int(raceId[12:14])
    placeId=raceId[9:11]
    #print(raceDate,raceNumber,placeId)
    html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/oddstf?rno=%d&jcd=%s&hd=%s" %(raceNumber,placeId,raceDate) )
    time.sleep(1)
    soup = BeautifulSoup(html,"html.parser")
    
    try:
        block =((soup.select(".grid_unit"))[0].select("table")[0]).select(".oddsPoint")
        i=1
        for tds in block:
            yield i,tds.string
            i+=1
    except IndexError as e:
        print('[SKIP]IndexError発生 at 単勝オッズ取得　raceId:'+raceId )
    
#raceId="20190321-01-01"
#for ret in getOldOddsKoushikiTansho(raceId):
#    print(ret)
    
raceDate="20190321"
dbh.rollback()

with dbh.cursor() as cursor:
    #racemanagementで単勝のオッズを埋める
    sel_sql = "SELECT raceId FROM racemanagement WHERE raceDate='%s' and raceodds1t_flg = FALSE" % (raceDate)
    cursor.execute(sel_sql)
    raceIdList=cursor.fetchall()
    for raceIdhash in raceIdList:
        raceId = raceIdhash['raceId']
        print(raceId)
        with dbh.cursor() as ins_cursor:
            for ret in getOldOddsKoushikiTansho(raceId):
                #print(ret)
                if ret[1] == '欠場':
                    print('[SKIP]欠場 at 単勝オッズ取得 raceId:'+raceId)
                else:
                    ins_sql="INSERT INTO raceodds (raceId,oddsType,funaken,odds) VALUES ('%s' ,'1t','%s','%s')" % (raceId,ret[0],ret[1])
                    ins_cursor.execute(ins_sql)
            upd_sql = "UPDATE racemanagement SET raceodds1t_flg = TRUE where raceId ='%s'" % (raceId)
            ins_cursor.execute(upd_sql)
            dbh.commit()

            

20190321-01-01
20190321-01-02
20190321-01-03
20190321-01-04
20190321-01-05
20190321-01-06
20190321-01-07
20190321-01-08
20190321-01-09
20190321-01-10
20190321-01-11
20190321-01-12
20190321-02-01
20190321-02-02
20190321-02-03
20190321-02-04
20190321-02-05
20190321-02-06
20190321-02-07
20190321-02-08
20190321-02-09
20190321-02-10
20190321-02-11
20190321-02-12
20190321-03-01
[SKIP]IndexError発生 at 単勝オッズ取得　raceId:20190321-03-01
20190321-03-02
[SKIP]IndexError発生 at 単勝オッズ取得　raceId:20190321-03-02
20190321-03-03
[SKIP]IndexError発生 at 単勝オッズ取得　raceId:20190321-03-03
20190321-03-04
[SKIP]IndexError発生 at 単勝オッズ取得　raceId:20190321-03-04
20190321-03-05
[SKIP]IndexError発生 at 単勝オッズ取得　raceId:20190321-03-05
20190321-03-06
[SKIP]IndexError発生 at 単勝オッズ取得　raceId:20190321-03-06
20190321-03-07
[SKIP]IndexError発生 at 単勝オッズ取得　raceId:20190321-03-07
20190321-03-08
[SKIP]IndexError発生 at 単勝オッズ取得　raceId:20190321-03-08
20190321-03-09
[SKIP]IndexError発生 at 単勝オッズ取得　raceId:20190321-03-09
20190321-03-10
[SKIP]IndexError発生 at 

In [27]:
# 複勝は使わない。形式が難しいから後回し。
# def getOldOddsKoushikiFukusho(raceId):
#     raceDate=raceId[0:8]
#     raceNumber=int(raceId[12:14])
#     placeId=raceId[9:11]
#     #print(raceDate,raceNumber,placeId)
#     html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/oddstf?rno=%d&jcd=%s&hd=%s" %(raceNumber,placeId,raceDate) )
#     time.sleep(1)
#     soup = BeautifulSoup(html,"html.parser")
    
#     try:
#         block =((soup.select(".grid_unit"))[1].select("table")[0]).select(".oddsPoint")
#         i=1
#         for tds in block:
#             yield i,tds.string
#             i+=1
#     except IndexError as e:
#         print('[SKIP]IndexError発生 at 複勝オッズ取得　raceId:'+raceId )
    
# raceDate="20190321"
# dbh.rollback()

# with dbh.cursor() as cursor:
#     #racemanagementで複勝のオッズを埋める
#     sel_sql = "SELECT raceId FROM racemanagement WHERE raceDate='%s' and raceodds1f_flg = FALSE" % (raceDate)
#     cursor.execute(sel_sql)
#     raceIdList=cursor.fetchall()
#     for raceIdhash in raceIdList:
#         raceId = raceIdhash['raceId']
#         print(raceId)
#         with dbh.cursor() as ins_cursor:
#             for ret in getOldOddsKoushikiFukusho(raceId):
#                 #print(ret)
#                 if ret[1] == '欠場':
#                     print('[SKIP]欠場 at 複勝オッズ取得 raceId:'+raceId)
#                 else:
#                     ins_sql="INSERT INTO raceodds (raceId,oddsType,funaken,odds) VALUES ('%s' ,'1f','%s','%s')" % (raceId,ret[0],ret[1])
#                     ins_cursor.execute(ins_sql)
#             upd_sql = "UPDATE racemanagement SET raceodds1f_flg = TRUE where raceId ='%s'" % (raceId)
#             ins_cursor.execute(upd_sql)
#             dbh.commit()


In [64]:
def getOldOddsKoushikiResult(raceId):
    raceDate=raceId[0:8]
    raceNumber=int(raceId[12:14])
    placeId=raceId[9:11]
    #print(raceDate,raceNumber,placeId)
    html = u.urlopen("https://www.boatrace.jp/owpc/pc/race/raceresult?rno=%d&jcd=%s&hd=%s" %(raceNumber,placeId,raceDate) )
    time.sleep(1)
    soup = BeautifulSoup(html,"html.parser")

    try:
        blocks_time =(soup.select(".is-w495"))[0].select("tr")
        blocks_start =(soup.select(".is-w495"))[1].select("tr")
        #print(blocks_start)
        del blocks_time[0] # 1行目のヘッダは削除
        del blocks_start[0] # 1行目のヘッダは削除

        tds_start=[]
        for block_start in blocks_start:
            tmp_starttime=((block_start.select(".table1_boatImage1TimeInner"))[0].string.split(' '))[0]
            if tmp_starttime[0]=='F':
                tmp_starttime='-0'+tmp_starttime[1:]
            else:
                tmp_starttime='0'+tmp_starttime
            tds_start.append(tmp_starttime.strip())
            
        for i in range(6):
            block_time=blocks_time[i]
            tds_time = block_time.select("td")
            #print(tds)
            print("waku"+tds_time[1].string)
            print("sttime"+tds_start[int(tds_time[1].string)-1])
            print("racer"+(tds_time[2].select(".is-fs12"))[0].string )
            print("time"+tds_time[3].string)
            #print(block)
    except IndexError as e:
        print('[SKIP]IndexError発生 at 結果取得 raceid:'+raceId)
    
raceId="20190321-01-01"
getOldOddsKoushikiResult(raceId)
#for ret in getOldOddsKoushikiResult(raceId):
#    print(ret)



waku1
sttime0.12
racer3812
time1'51"2
waku3
sttime0.14
racer3261
time1'52"9
waku6
sttime0.18
racer3673
time1'55"0
waku4
sttime0.15
racer3290
time1'55"6
waku2
sttime0.20
racer3123
time1'57"0
waku5
sttime0.18
racer5063
time1'58"5
